In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import markdown

# Define the base URL and the number of pages to scrape
base_url = "https://lawrato.com/free-legal-advice"
num_pages = 10

# Define the keys for the JSON file
keys = ['title', 'content']

# Define the function to convert HTML tables to markdown
def html_to_markdown(html):
    return markdown.markdown(html)

# Define the function to scrape the website
def scrape_website():
    # Create an empty set to store the scraped data
    data = set()

    # Set up the Selenium driver
    options = webdriver.ChromeOptions()
    #options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    # Loop through the pages to scrape
    for i in range(1, num_pages+1):
        url = base_url + "?page=" + str(i)

        # Load the URL in the browser
        driver.get(url)
        time.sleep(5)
         
        j=5
        # Loop through the questions and extract the title and content
        for j in range(5, 15):
            try:
                title_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="guide-detail"]/div/div/div[2]/div[1]/div[{j}]/div[1]/div[2]/a/h2')))
                content_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="guide-detail"]/div/div/div[2]/div[1]/div[{j}]/div[2]/p[1]')))
            except:
                print(f"Warning: Failed to extract data from question on page {i}")
                continue

            title = title_elem.text.strip()
            content_html = content_elem.get_attribute('innerHTML').strip()

            # Convert the HTML content to plain text
            soup = BeautifulSoup(content_html, 'html.parser')
            content = soup.get_text().strip()

            # Add the title and content to the data set
            data.add((title, content))

    # Convert the set to a list and write the data to a JSON file
    data_list = [{'title': title, 'content': content} for title, content in data]
    with open('data.json', 'w') as f:
        json.dump(data_list, f)

    # Quit the browser
    driver.quit()

# Call the scrape_website function
scrape_website()